In [ ]:
import pandas as pd
import requests
import json
import time
import configparser
import datetime 

# Read the config file
config = configparser.ConfigParser()
config.read('tokens/config.ini')

## Get the tokens from file to connect to Strava
with open('tokens/strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
## If access_token has expired then use the refresh_token to get the new access_token
if strava_tokens['expires_at'] < time.time():
#Make Strava auth API call with current refresh token
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': int(config['STRAVA_TOKENS']['client_id']),
                                'client_secret': config['STRAVA_TOKENS']['client_secret'],
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    )
# Save response as json in new variable
    new_strava_tokens = response.json()
# Save new tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
#Use new Strava tokens from now
    strava_tokens = new_strava_tokens
#Loop through all activities
page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
## Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame(
    columns = [
            "name",
            "start_date_local",
            "type",
            "distance",
            "moving_time",
            "elapsed_time",
            "total_elevation_gain"

    ]
)
while True:
    
    # get page of activities from Strava
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()
# if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*200,'name'] = r[x]['name']
        activities.loc[x + (page-1)*200,'start_date_local'] = r[x]['start_date_local']
        activities.loc[x + (page-1)*200,'type'] = r[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = r[x]['distance']
        activities.loc[x + (page-1)*200,'moving_time'] = r[x]['moving_time']
        activities.loc[x + (page-1)*200,'elapsed_time'] = r[x]['elapsed_time']
        activities.loc[x + (page-1)*200,'total_elevation_gain'] = r[x]['total_elevation_gain']
# increment page
    page += 1
activities.to_csv('strava_activities_raw.csv')

In [ ]:
import datetime

def seconds_to_duration(n): 
    return str(datetime.timedelta(seconds = n)) 

act = pd.read_csv('strava_activities_raw.csv').drop(['Unnamed: 0'],axis=1)
act.rename(columns={'distance':'distance_meters'},inplace=True)
act['distance_miles'] = act.distance_meters / 1609
act['total_elevation_gain'] = act.total_elevation_gain * 3.28084
act['moving_time'] = act.apply(lambda x: seconds_to_duration(x['moving_time']),axis=1) #Converts seconds to duration
act['elapsed_time'] = act.apply(lambda x: seconds_to_duration(x['elapsed_time']),axis=1) #Converts seconds to duration

act.to_csv('strava_activities.csv')